### Minimal ensemble code

This code builds a simple bagging ensemble based on Kfold train/val split. Use it for quick idea testing with GPT-5-assisted tree generation.

In [1]:
# Local dataset settings
DATA_PATH = "mydata/data.csv"
TARGET_COL = "PM2.5"
TASK_TYPE = "regression"  # regression | binary | multiclass
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
N_REPEATS = 1

dataset_name = "pm25_local"
results_path = "data/tree_scores.pm25_local.json"

AUTH_SCHEME = "Bearer"  # e.g., DeepSeek uses Bearer; set to "OAuth" if required

In [2]:
from Token import *

In [3]:
import os
import re
import json
import numpy as np
import pandas as pd
import smolagents
from huggingface_hub import login
import proxy_api_model
import prompting
import tree_agent
from sklearn.model_selection import train_test_split
from task import metric_func_by_task
from smolagents import LiteLLMModel

# Debug: confirm which proxy_api_model is loaded
print("proxy_api_model file:", proxy_api_model.__file__)

login(token=HF_TOKEN_HERE)
# Use the OpenAI-like proxy API wrapper (supports callback)
model = LiteLLMModel(
    model_id="deepseek-reasoner",
    api_base=API_ENDPOINT_HERE,  # <-- https://api.deepseek.com/v1/chat/completions
    api_key=API_TOKEN_HERE,  # <-- raw sk- token only
    max_new_tokens=1024 * 8,
    verify_ssl=True,
    timeout=120,
    auth_scheme=AUTH_SCHEME,
    # callback=lambda msg, **etc: print(  # print model thoughts before code
    #     re.sub(r"<code>.*?</code>", "<code omitted>", msg.content, flags=re.DOTALL)
    # ),
)

# Load local dataset
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"DATA_PATH not found: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)

if TARGET_COL not in df.columns:
    matches = [c for c in df.columns if c.lower() == TARGET_COL.lower()]
    if matches:
        TARGET_COL = matches[0]
    else:
        raise ValueError(
            f"Target column '{TARGET_COL}' not found in data columns: {list(df.columns)}"
        )

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].to_numpy()

task_type = TASK_TYPE
metric_func = metric_func_by_task[task_type]
metric_name = prompting.metrics_by_task[task_type]

cat_cols = [c for c in X.columns if X[c].dtype == "object"]
num_cols = [c for c in X.columns if c not in cat_cols]

# Dataset description to help the LLM form hypotheses
feature_list = ", ".join(X.columns)
cat_list = ", ".join(cat_cols) if cat_cols else "none"
num_list = ", ".join(num_cols) if num_cols else "none"

dataset_desc = f"""
<dataset>
Your task is to predict PM2.5.
Size: {len(X)} total (full dataset)
Feature columns: {feature_list}
Categorical columns: {cat_list}
Numeric columns: {num_list}
Targets: {TARGET_COL} (float)
Metric: {metric_name}
</dataset>
""".strip()

c:\Users\CaoBo\anaconda3\envs\main\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


proxy_api_model file: c:\Users\CaoBo\OneDrive\VScode\Python_workspace\Python_Project_B\TalkingTrees\proxy_api_model.py


In [4]:
import os
import requests

# Ensure API config is available even if earlier cells were not run
if "API_ENDPOINT_HERE" not in globals():
    API_ENDPOINT_HERE = (
        os.getenv("API_ENDPOINT_HERE") or os.getenv("API_ENDPOINT") or ""
    )
if "API_TOKEN_HERE" not in globals():
    API_TOKEN_HERE = os.getenv("API_TOKEN_HERE") or os.getenv("API_TOKEN") or ""
if not API_ENDPOINT_HERE or not API_TOKEN_HERE:
    raise ValueError(
        "Missing API config. Run the config cell or set API_ENDPOINT_HERE/API_TOKEN_HERE env vars."
    )

API_ENDPOINT = API_ENDPOINT_HERE
API_TOKEN = API_TOKEN_HERE

r = requests.post(
    API_ENDPOINT,
    headers={
        "authorization": f"Bearer {API_TOKEN}",
        "content-type": "application/json",
    },
    json={
        "model": "deepseek-reasoner",
        "messages": [{"role": "user", "content": "ping"}],
        "max_tokens": 1,
    },
    timeout=30,
)
print("status:", r.status_code)
print("body:", r.text[:300])


test_scores = []
r2_scores = []
rmse_scores = []
last_y_test = None
last_y_pred = None
for repeat_index in range(N_REPEATS):
    print("Beginning repeat", repeat_index)
    split_seed = RANDOM_STATE if N_REPEATS == 1 else RANDOM_STATE + repeat_index

    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X,
        y,
        test_size=TEST_SIZE,
        random_state=split_seed,
        stratify=None if task_type == "regression" else y,
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full,
        y_train_full,
        test_size=VAL_SIZE,
        random_state=split_seed,
        stratify=None if task_type == "regression" else y_train_full,
    )

    result = tree_agent.TreeAgent(model=model).run(
        task=f"""
Build the optimal decision tree for the '{dataset_name}' dataset.
You are given access to 4 data variables in your python environment:
 - X_train, X_val are pandas dataframes with named feature columns (see below) that may need preprocessing;
 - y_train, y_val are numpy arrays (1d) with targets, also described below;

Dataset description (use it to form hypotheses):
{dataset_desc}

Here's one way you could construct before you begin editing it manually:
{prompting.starter_snippets_by_task[task_type]}

Now begin: view the data variables, preprocess as necessary, train a baseline tree, then propose the first hypothesis and start improving.
Focus on drawing conclusions from data, looking at the tree (e.g. via print) and using your own intuition about the problem for manual tree edits.
Quality is more important than speed: take as many steps as you need to get the best tree.
""".strip(),
        additional_args=dict(
            X_train=X_train.copy(),
            y_train=y_train.copy(),
            X_val=X_val.copy(),
            y_val=y_val.copy(),
        ),
    )

    y_pred_i = result["model"].predict(result["preprocess_features"](X_test.copy()))
    if task_type == "multiclass":  # normalize for logloss
        y_pred_i = y_pred_i / y_pred_i.sum(axis=-1, keepdims=True)
    test_score = metric_func(y_test, y_pred_i)
    if task_type == "regression":
        from sklearn.metrics import r2_score, root_mean_squared_error

        r2 = r2_score(y_test, y_pred_i)
        rmse = root_mean_squared_error(
            y_test,
            y_pred_i,
        )
        print(f"Test R2 score #{repeat_index}: {r2:.5f}")
        print(f"Test RMSE score #{repeat_index}: {rmse:.5f}")
        r2_scores.append(r2)
        rmse_scores.append(rmse)
    print(f"Test {metric_name} score #{repeat_index}: {test_score:.5f}")
    test_scores.append(test_score)
    with open(results_path, "w") as f:
        json.dump(test_scores, f, ensure_ascii=False, indent=2)

status: 200
body: {"id":"0218ecd8-f0b0-446a-99c8-c90e2f6fd144","object":"chat.completion","created":1770648586,"model":"deepseek-reasoner","choices":[{"index":0,"message":{"role":"assistant","content":"","reasoning_content":"H"},"logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":5,"completion_tokens"
Beginning repeat 0


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Build the optimal decision tree for the 'pm25_local' dataset.                                                   │
│ You are given access to 4 data variables in your python environment:                                            │
│  - X_train, X_val are pandas dataframes with named feature columns (see below) that may need preprocessing;     │
│  - y_train, y_val are numpy arrays (1d) with targets, also described below;                                     │
│                                                                                                                 │
│ Dataset description (use it to form hypotheses):                                                                │
│ <dataset>                                                                                                       │
│ Your task is to predict PM2.5.                                                                                  │
│ Size: 2676 total (full dataset)                                                                                 │
│ Feature columns: OBJECTID_1, PROVINCE, CITY, COUNTY, AET, ppt, tem, wind, NOX, SO2, fertilzier, manure          │
│ Categorical columns: PROVINCE, CITY, COUNTY                                                                     │
│ Numeric columns: OBJECTID_1, AET, ppt, tem, wind, NOX, SO2, fertilzier, manure                                  │
│ Targets: PM2.5 (float)                                                                                          │
│ Metric: RMSE (sqrt of sklearn.metrics.mean_squared_error)                                                       │
│ </dataset>                                                                                                      │
│                                                                                                                 │
│ Here's one way you could construct before you begin editing it manually:                                        │
│ <code>                                                                                                          │
│ import numpy as np                                                                                              │
│ import pandas as pd                                                                                             │
│ from sklearn.tree import DecisionTreeRegressor                                                                  │
│ from sklearn.metrics import mean_squared_error                                                                  │
│                                                                                                                 │
│ # you are given input data: X_train/val are pd.DataFrames, y_train/val are 1d np.ndarrays                       │
│ X_train, X_val, y_train, y_val = preprocess_data_somehow(X_train, X_val, y_train, y_val)  # <-- part of your    │
│ task                                                                                                            │
│                                                                                                                 │
│ tree = DecisionTreeRegressor(criterion="squared_error", min_samples_split=100)                                  │
│ tree.fit(X_train, y_train)  # important: the tree is trained on a pd.DataFrame                                  │
│ print("Train RMSE:", np.sqrt(mean_squared_error(y_train, tree.predict(X_train))))                               │
│ print("Val RMSE:", np.sqrt(mean_squared_error(y_val, tree.predict(X_val))))                                     │
│ from editable_tree import Tree                                                                                  │
│ print(Tree.from_sklearn(tree))                        

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Provider List: https://docs.litellm.ai/docs/providers



─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
                                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...


Provider List: https://docs.litellm.ai/docs/providers



─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "summary": "Final decision tree for PM2.5 prediction with log-transformed features and target-encoded        
  categories. The tree was built with max_depth=4 and min_samples_split=50, and manually adjusted to correct an    
  outlier leaf (leaf 8) that had a single sample with an extreme value, setting it to the mean of its sibling      
  node (26.431). The tree splits primarily on log_NOX and log_SO2, with additional splits on CITY, PROVINCE,       
  temperature, and log_manure. The model achieves a validation RMSE of 13.1661.",                                  
      "train_score": 11.3907,                                                                                      
      "val_score": 13.1661,                                                                                        
      "model": final_tree,                                                                                         
      "preprocess_features": preprocess_features_final                                                             
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'summary': 'Final decision tree for PM2.5 prediction with log-transformed features and 
target-encoded categories. The tree was built with max_depth=4 and min_samples_split=50, and manually adjusted to 
correct an outlier leaf (leaf 8) that had a single sample with an extreme value, setting it to the mean of its 
sibling node (26.431). The tree splits primarily on log_NOX and log_SO2, with additional splits on CITY, PROVINCE, 
temperature, and log_manure. The model achieves a validation RMSE of 13.1661.', 'train_score': 11.3907, 
'val_score': 13.1661, 'model':  if log_NOX <= 3.173:  # id: 0, then id: 1, else id: 12, value: 32.314
  then: if log_SO2 <= 0.629:  # id: 1, then id: 2, else id: 7, value: 25.589
    then: if CITY <= 31.656:  # id: 2, then id: 3, else id: 6, value: 19.967
      then: if PROVINCE <= 33.912:  # id: 3, then id: 4, else id: 5, value: 13.458
        then: leaf value=10.539  # id: 4
        else: leaf value=31.387  # id: 5
      else: leaf value=27.199  # id: 6
    else: if log_SO2 <= 0.629:  # id: 7, then id: 8, else id: 9, value: 26.524
      then: leaf value=26.431  # id: 8
      else: if log_manure <= 8.152:  # id: 9, then id: 10, else id: 11, value: 26.431
        then: leaf value=28.816  # id: 10
        else: leaf value=25.356  # id: 11
  else: if log_SO2 <= 2.816:  # id: 12, then id: 13, else id: 18, value: 36.596
    then: if log_SO2 <= 1.589:  # id: 13, then id: 14, else id: 15, value: 31.238
      then: leaf value=16.349  # id: 14
      else: if log_NOX <= 3.181:  # id: 15, then id: 16, else id: 17, value: 31.671
        then: leaf value=48.615  # id: 16
        else: leaf value=31.385  # id: 17
    else: if CITY <= 28.002:  # id: 18, then id: 19, else id: 22, value: 38.26
      then: if CITY <= 25.694:  # id: 19, then id: 20, else id: 21, value: 31.657
        then: leaf value=36.953  # id: 20
        else: leaf value=28.264  # id: 21
      else: if tem <= 19.492:  # id: 22, then id: 23, else id: 24, value: 39.261
        then: leaf value=38.928  # id: 23
        else: leaf value=54.32  # id: 24, 'preprocess_features': <function create_function.<locals>.new_func at 
0x000001570DA60A40>}

[Step 18: Duration 11.18 seconds| Input tokens: 480,437 | Output tokens: 23,941]

Test R2 score #0: 0.21172
Test RMSE score #0: 12.12085
Test RMSE (sqrt of sklearn.metrics.mean_squared_error) score #0: 12.12085


In [5]:
# Summary outputs
print(f"Final {metric_name} score list: {test_scores}")
if len(test_scores) > 0:
    print(f"Mean {metric_name}: {np.mean(test_scores):.5f}")
    if len(test_scores) > 1:
        print(f"Std {metric_name}: {np.std(test_scores, ddof=1):.5f}")

if task_type == "regression":
    if len(r2_scores) > 0:
        print(f"Mean R2: {np.mean(r2_scores):.5f}")
        if len(r2_scores) > 1:
            print(f"Std R2: {np.std(r2_scores, ddof=1):.5f}")
    if len(rmse_scores) > 0:
        print(f"Mean RMSE: {np.mean(rmse_scores):.5f}")
        if len(rmse_scores) > 1:
            print(f"Std RMSE: {np.std(rmse_scores, ddof=1):.5f}")

    if last_y_test is not None and last_y_pred is not None:
        from sklearn.metrics import r2_score

        try:
            from sklearn.metrics import root_mean_squared_error as _rmse

            final_rmse = _rmse(last_y_test, last_y_pred)
        except Exception:
            from sklearn.metrics import mean_squared_error

            final_rmse = mean_squared_error(last_y_test, last_y_pred) ** 0.5
        final_r2 = r2_score(last_y_test, last_y_pred)
        print(f"Final R2: {final_r2:.5f}")
        print(f"Final RMSE: {final_rmse:.5f}")

Final RMSE (sqrt of sklearn.metrics.mean_squared_error) score list: [12.120850081058729]
Mean RMSE (sqrt of sklearn.metrics.mean_squared_error): 12.12085
Mean R2: 0.21172
Mean RMSE: 12.12085
